In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Đọc và tiền xử lý dữ liệu
data_file_path = 'data_goc.csv'
data = pd.read_csv(data_file_path)

data['priceVnd'] = pd.to_numeric(data['priceVnd'], errors='coerce')
data['areaM2'] = pd.to_numeric(data['areaM2'], errors='coerce')
data['lat'] = pd.to_numeric(data['lat'], errors='coerce')
data['long'] = pd.to_numeric(data['long'], errors='coerce')
data.dropna(subset=['priceVnd', 'areaM2', 'lat', 'long'], inplace=True)

X = data[['areaM2', 'lat', 'long']].values
y = np.log1p(data['priceVnd'].values)

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Định dạng dữ liệu cho từng mô hình
X_train_cnn = X_train.reshape((X_train.shape[0], 1, 1, X_train.shape[1]))
X_test_cnn = X_test.reshape((X_test.shape[0], 1, 1, X_test.shape[1]))

X_train_rnn = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_rnn = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_17532\3190893783.py:17: RuntimeWarning: divide by zero encountered in log1p
  y = np.log1p(data['priceVnd'].values)


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model_cnn = Sequential()

# Lớp Convolutional và MaxPooling
model_cnn.add(Conv2D(32, (1, 1), activation='relu', input_shape=(1, 1, X_train.shape[1])))
model_cnn.add(MaxPooling2D((1, 1)))

# Lớp Convolutional tiếp theo
model_cnn.add(Conv2D(64, (1, 1), activation='relu'))
model_cnn.add(MaxPooling2D((1, 1)))

# Lớp Convolutional thứ ba
model_cnn.add(Conv2D(128, (1, 1), activation='relu'))
model_cnn.add(MaxPooling2D((1, 1)))

# Lớp Flatten
model_cnn.add(Flatten())

# Dense Layers
model_cnn.add(Dense(128, activation='relu'))
model_cnn.add(Dropout(0.5))
model_cnn.add(Dense(64, activation='relu'))
model_cnn.add(Dropout(0.5))

# Lớp đầu ra
model_cnn.add(Dense(1))

# Compile và huấn luyện
model_cnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model_cnn.fit(X_train_cnn, y_train, validation_split=0.2, epochs=50, batch_size=32)


c:\Users\ADMIN\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 434ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: nan - mae: nan

In [4]:
from tensorflow.keras.layers import SimpleRNN

model_rnn = Sequential()

# Lớp RNN đầu tiên
model_rnn.add(SimpleRNN(128, input_shape=(1, X_train.shape[1]), return_sequences=True))
model_rnn.add(Dropout(0.5))

# Các lớp RNN tiếp theo
for _ in range(5):
    model_rnn.add(SimpleRNN(128, return_sequences=True))
    model_rnn.add(Dropout(0.5))

# Lớp đầu ra
model_rnn.add(Dense(1))

# Compile và huấn luyện
model_rnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model_rnn.fit(X_train_rnn, y_train, validation_split=0.2, epochs=50, batch_size=32)


c:\Users\ADMIN\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: nan - mae: nan 

In [5]:
from tensorflow.keras.layers import LSTM

model_lstm = Sequential()

# Lớp LSTM đầu tiên
model_lstm.add(LSTM(128, input_shape=(1, X_train.shape[1]), return_sequences=True))
model_lstm.add(Dropout(0.5))

# Các lớp LSTM tiếp theo
for _ in range(5):
    model_lstm.add(LSTM(128, return_sequences=True))
    model_lstm.add(Dropout(0.5))

# Lớp đầu ra
model_lstm.add(Dense(1))

# Compile và huấn luyện
model_lstm.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model_lstm.fit(X_train_rnn, y_train, validation_split=0.2, epochs=50, batch_size=32)


Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 21s 3s/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: nan - mae: nan 